In [ ]:
# 1. Import the libraries
import requests
import json
import subprocess

# 2. Add your Azure OpenAI account details
subscription = "<AOAI-ACCOUNT-SUBSCRIPTION-ID>"
resource_group = "<AOAI-ACCOUNT-RESOURCE-GROUP>"
resource_name = "<AOAI-RESOURCE-NAME>"
model_deployment_name = "<NEW-AOAI-DEPLOYMENT-NAME>"
registered_model_name = 'gpt-4o'
registered_model_version = '2024-05-13'

In [ ]:
# 2. Add your Azure OpenAI account details
subscription = "<AOAI-ACCOUNT-SUBSCRIPTION-ID>"
resource_group = "<AOAI-ACCOUNT-RESOURCE-GROUP>"
resource_name = "<AOAI-RESOURCE-NAME>"
model_deployment_name = "<NEW-AOAI-DEPLOYMENT-NAME>"
registered_model_name = 'gpt-4o'
registered_model_version = '2024-05-13'

In [ ]:
# 3. Get the access token which will be passed during PUT request call.
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    access_token = credential.get_token("https://management.azure.com/.default")
    token = access_token.token
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    # This will open a browser page for
    credential = InteractiveBrowserCredential()
    
    
deploy_params = {"api-version": "2024-04-01-preview"}
deploy_headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json",
}

In [ ]:
# 4. Set model deployment configuration. Here capacity refers to support for `1K Tokens Per Minute (TPM)` for your deployment.
deploy_data = {
    "displayName": model_deployment_name,
    "properties": {
        "model": {
            "format": "OpenAI",
            "name": registered_model_name,
            "version": registered_model_version
        },
        "versionUpgradeOption": "OnceNewDefaultVersionAvailable",
        "dynamicThrottlingEnabled": "true",
        "raiPolicyName": "Microsoft.Default"
    },
    "sku": {
        "name": "Standard",
        "capacity": 150
    }
}

In [ ]:
# 4. Call the Request and Track the deployment

request_url = f"https://management.azure.com//subscriptions/{subscription}/resourceGroups/{resource_group}/providers/Microsoft.CognitiveServices/accounts/{resource_name}/deployments/{model_deployment_name}"
r = requests.put(   
    request_url, 
    params=deploy_params, 
    headers=deploy_headers, 
    data=deploy_data
)

print(r.json())

deploy_data = json.dumps(deploy_data)